## Import Libraries

In [ ]:
import os
import math
import random

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
from keras.regularizers import l2

In [ ]:
tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

## Seed

In [ ]:
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

## Model Parameters

In [ ]:
# All the categories to classify the dataset
labels = ['Apple',              # 0
          'Blueberry',          # 1
          "Cherry",             # 2
          "Corn",               # 3
          "Grape",              # 4
          "Orange",             # 5
          "Peach",              # 6
          "Pepper",             # 7
          "Potato",             # 8
          "Raspberry",          # 9
          "Soybean",            # 10
          "Squash",             # 11
          "Strawberry",         # 12
          "Tomato"]             # 13

In [ ]:
img_w = 256
img_h = 256
input_shape = (256, 256, 3)
classes = 14

class_weight = {
    0: 1.2816657027183342, 
    1: 2.711532578770266, 
    2: 2.1720166625827004, 
    3: 1.0499881544657663, 
    4: 0.8685087203605723, 
    5: 0.7244197450147107, 
    6: 1.296095920456207, 
    7: 1.6552754435107375, 
    8: 1.7685554668794892, 
    9: 4.796536796536796, 
    10: 0.7835926449787836, 
    11: 2.2060726729716276, 
    12: 1.8815538102313734, 
    13: 0.22242854633509823
}

epochs = 30
patience_epochs = 8
batch_size = 24

last_nonTrainable_layer = 207

In [ ]:
# Dataset folders 
dataset_dir = '../datasetNoTest'
training_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'val')
#test_dir = os.path.join(dataset_dir, 'test')

## Data Augmentation

In [ ]:
train_data_gen = ImageDataGenerator(rotation_range=20,
                                        height_shift_range=0.3,
                                        width_shift_range=0.4,
                                        zoom_range=0.4,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        brightness_range=[0.3,1.4],
                                        fill_mode='nearest',
                                        preprocessing_function=preprocess_input)

train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=labels,
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed)

In [ ]:
valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=labels,
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=False,
                                               seed=seed)

## Transfer Learning Model

In [ ]:
# Download and plot the InceptionV3 model
supernet = tfk.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3)
)

supernet.trainable = True

for i, layer in enumerate(supernet.layers[:last_nonTrainable_layer]):
  layer.trainable=False

for i, layer in enumerate(supernet.layers):
   print(i, layer.name, layer.trainable)

## Learning Rate Scheduler

In [ ]:
def step_decay(epoch):

   initial_lrate = 0.005
   drop = 0.1
   epochs_drop = 10.0

   lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))

   return lrate

## Callbacks

In [ ]:
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('callbackSaves')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=True) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)


  # Early Stopping
  # --------------
  #es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='max', restore_best_weights=True)
  #callbacks.append(es_callback)


  # Learning Rate Scheduler
  # --------------
  LRS_callback = tf.keras.callbacks.LearningRateScheduler(step_decay)
  callbacks.append(LRS_callback)
  

  return callbacks

## Network Model

In [ ]:
# Use the supernet as feature extractor

inputs = tfk.Input(shape=input_shape)

x = supernet(inputs)

glob_pooling = tfkl.GlobalAveragePooling2D(name='GlobalPooling')(x)

x = tfkl.Dense(
    512,
    kernel_initializer = tfk.initializers.GlorotUniform(seed)
)(glob_pooling)

leaky_relu_layer = tfkl.LeakyReLU()(x)

x = tfkl.Dropout(0.3, seed=seed)(leaky_relu_layer)

outputs = tfkl.Dense(
    classes, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed),
)(x)


# Connect input and output through the Model class
ft_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.SGD(momentum=0.9, decay=0.0005, nesterov=False), metrics=['accuracy', tfk.metrics.Precision(), tfk.metrics.Recall()])
ft_model.summary()

## Training

In [ ]:
callbacks = create_folders_and_callbacks(model_name='GoogleNetModel')

history = ft_model.fit(
    x = train_gen,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = valid_gen,
    class_weight = class_weight,
    callbacks = callbacks
).history

ft_model.save("fineTuningModel")

## Plots

In [ ]:
ALPHA = 0.5

plt.figure(figsize=(20,10))

plt.plot(history['accuracy'], label='Accuracy Train', alpha=ALPHA, color='#E64A19', linestyle='--')
plt.plot(history['val_accuracy'], label='Accuracy Val', alpha=ALPHA, color='#F57C00')

plt.plot(history['precision_8'], label='Precision Train', alpha=ALPHA, color='#388E3C', linestyle='--')
plt.plot(history['val_precision_8'], label='Precision Val', alpha=ALPHA, color='#689F38')

plt.plot(history['recall_8'], label='Recall Train', alpha=ALPHA, color='#303F9F', linestyle='--')
plt.plot(history['val_recall_8'], label='Recall Val', alpha=ALPHA, color='#1976D2')

plt.ylim(.9, 1)
plt.title('Metrics')
plt.legend(loc='lower right')
plt.grid(alpha=.3)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history['loss'], label='loss', alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(history['val_loss'], label='val_loss', alpha=.8, color='#2ABC3D')

plt.ylim(0, .1)
plt.legend(loc='upper right')
plt.title('Loss')
plt.grid(alpha=.3)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history['accuracy'], label='Accuracy Train', alpha=ALPHA, color='#E64A19', linestyle='--')
plt.plot(history['val_accuracy'], label='Accuracy Val', alpha=ALPHA, color='#F57C00')

plt.ylim(.9, 1)
plt.legend(loc='lower right')
plt.title('Accuracy')
plt.grid(alpha=.3)

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history['precision_8'], label='Precision Train', alpha=ALPHA, color='#388E3C', linestyle='--')
plt.plot(history['val_precision_8'], label='Precision Val', alpha=ALPHA, color='#689F38')

plt.ylim(.9, 1)
plt.legend(loc='lower right')
plt.title('Precision')
plt.grid(alpha=.3)

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history['recall_8'], label='Recall Train', alpha=ALPHA, color='#303F9F', linestyle='--')
plt.plot(history['val_recall_8'], label='Recall Val', alpha=ALPHA, color='#1976D2')

plt.legend(loc='lower right')
plt.title('Recall')
plt.grid(alpha=.3)

## Testing

In [ ]:
#model_test_metrics = ft_model.evaluate(test_gen, return_dict=True)
#print(model_test_metrics)